In [1]:
# --- Cell 1: Install Ultralytics (YOLOv8) ---
!pip install ultralytics
print(">>> Ultralytics installed successfully.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.2 MB/s eta 0:00:00
>>> Ultralytics installed successfully.


In [2]:
# --- Cell 2: Unzip Dataset & Fix Paths ---
import os
import shutil
from pathlib import Path

print(">>> [Step 2/5] Unzipping dataset...")
zip_file_name = "Dental Caries X-ray.v6199i.yolov8-obb.zip"
unzip_dir = Path("/content/dataset")
sub_dir_name = "Dental Caries X-ray.v6199i.yolov8-obb"
sub_dir = unzip_dir / sub_dir_name

!unzip -q "{zip_file_name}" -d "{unzip_dir}"

print(f">>> Moving files from {sub_dir} to {unzip_dir}...")
if sub_dir.exists() and sub_dir.is_dir():
    try:
        for item in sub_dir.iterdir():
            shutil.move(str(item), str(unzip_dir / item.name))
        sub_dir.rmdir()
        print(">>> Dataset unzipped and paths fixed.")
    except Exception as e:
        print(f"Error moving files: {e}. Check if files were already moved.")
else:
     print(f">>> Subdirectory {sub_dir_name} not found, assuming files are already in {unzip_dir}.")

>>> [Step 2/6] Unzipping dataset...
>>> Moving files from /content/dataset/Dental Caries X-ray.v6199i.yolov8-obb to /content/dataset...
>>> Dataset unzipped and paths fixed.


In [3]:
# --- Cell 3: Clean and MERGE Label Files ---
import os
from pathlib import Path

print(">>> [Step 3/5] Starting label MERGE process...")
NEW_CLASS_ID = '0'

label_dirs = [
    Path("/content/dataset/train/labels"),
    Path("/content/dataset/valid/labels"),
    Path("/content/dataset/test/labels")
]

file_count = 0
total_line_count = 0

for dir_path in label_dirs:
    print(f"--- Merging labels in {dir_path} ---")
    if not dir_path.exists():
        print(f"Warning: Directory not found {dir_path}, skipping.")
        continue

    for label_file in dir_path.glob("*.txt"):
        file_count += 1
        cleaned_lines = []
        try:
            with open(label_file, 'r') as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split()
                if not parts:
                    continue
                parts[0] = NEW_CLASS_ID
                cleaned_lines.append(" ".join(parts))
                total_line_count += 1

            with open(label_file, 'w') as f:
                f.write("\n".join(cleaned_lines))
        except Exception as e:
            print(f"Error processing file {label_file}: {e}")

print(f"--- Label Merging Finished! ---")
print(f"Processed {file_count} files.")
print(f"Found and MERGED {total_line_count} total labels into a single 'Caries' class.")

>>> [Step 3/6] Starting label MERGE process...
--- Merging labels in /content/dataset/train/labels ---
--- Merging labels in /content/dataset/valid/labels ---
--- Merging labels in /content/dataset/test/labels ---
--- Label Merging Finished! ---
Processed 1483 files.
Found and MERGED 2706 total labels into a single 'Caries' class.


In [4]:
# --- Cell 4: Create the Correct 1-Class data.yaml ---
from pathlib import Path

print("\n>>> [Step 4/5] Creating new 1-class data.yaml...")
yaml_path = Path("/content/dataset/data.yaml")

yaml_content = """
train: train/images
val: valid/images
test: test/images

nc: 1

names:
  - Caries
"""

try:
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)
    print(f">>> New data.yaml created at {yaml_path}")
except Exception as e:
    print(f"Error writing data.yaml: {e}")


>>> [Step 4/6] Creating new 1-class data.yaml...
>>> New data.yaml created at /content/dataset/data.yaml


In [3]:
# --- Cell 4.5: Clear GPU Memory (IMPORTANT!) ---
# Run this cell to clear any previous GPU memory usage
import torch
import gc

print(">>> Clearing GPU memory...")
torch.cuda.empty_cache()
gc.collect()
print(">>> GPU memory cleared!")

# Check available GPU memory
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f">>> Total GPU memory: {gpu_memory:.2f} GB")
    print(f">>> GPU: {torch.cuda.get_device_name(0)}")

>>> Clearing GPU memory...
>>> GPU memory cleared!
>>> Total GPU memory: 14.74 GB
>>> GPU: Tesla T4


In [4]:
# --- Cell 5: OPTIMIZED Training for T4 GPU ---
# This version is optimized to work within T4 GPU memory constraints
# while still providing significant improvements over the baseline

from ultralytics import YOLO

print("\n>>> [Step 5/5] Starting OPTIMIZED training for T4 GPU...")
print(">>> This version balances improvements with GPU memory constraints")

# OPTION 1: YOLOv8s with reduced settings (RECOMMENDED)
# This keeps the better model but reduces memory usage
model = YOLO('yolov8s.pt')
run_name = 'run_optimized_yolov8s'

# OPTION 2: If Option 1 still fails, uncomment these lines:
# model = YOLO('yolov8n.pt')
# run_name = 'run_optimized_yolov8n'

data_yaml_path = '/content/dataset/data.yaml'
project_name = 'caries_detection'

results = model.train(
    data=data_yaml_path,

    # --- Experiment Control ---
    project=project_name,
    name=run_name,

    # --- Training Parameters (OPTIMIZED FOR T4) ---
    epochs=200,               # Keep higher epochs
    batch=16,                 # REDUCED from 64 to 16 (critical for memory)
    imgsz=640,                # REDUCED from 800 to 640 (saves significant memory)
    patience=50,              # Keep higher patience
    cache=False,              # Don't cache images in memory

    # --- Optimizer Parameters (OPTIMIZED) ---
    optimizer='AdamW',
    lr0=0.001,                # Optimized learning rate
    lrf=0.001,                # Lower final LR
    weight_decay=0.0005,
    warmup_epochs=5,          # Longer warmup

    # --- Augmentation Control (ENHANCED but memory-friendly) ---
    degrees=15,
    translate=0.1,
    scale=0.6,
    flipud=0.5,
    fliplr=0.5,               # Horizontal flip
    hsv_h=0.015,              # Hue variation
    hsv_s=0.7,                # Saturation variation
    hsv_v=0.4,                # Brightness variation
    mosaic=1.0,               # Mosaic augmentation
    mixup=0.05,               # REDUCED from 0.1 (saves memory)
    copy_paste=0.0,           # DISABLED (saves memory)

    # --- Loss Weights ---
    box=7.5,
    cls=0.5,
    dfl=1.5,

    # --- Memory optimization ---
    amp=True,                 # Use automatic mixed precision
    workers=2                 # Reduce dataloader workers
)

print("--- OPTIMIZED Training Finished! ---")
print(f">>> Results saved to: /content/{project_name}/{run_name}")


>>> [Step 5/6] Starting OPTIMIZED training for T4 GPU...
>>> This version balances improvements with GPU memory constraints
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=15, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.001, mask_ratio=4, max_det=300, mixup=0.05, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run_optimize

In [5]:
# --- Cell 6: Download the Optimized Model ---
from google.colab import files
from pathlib import Path

print("\n>>> [Step 6/6] Starting download of the OPTIMIZED 'best.pt'...")

# Update this path based on which option you used in Cell 5
best_model_path_str = "/content/caries_detection/run_optimized_yolov8s/weights/best.pt"
# If you used Option 2 (yolov8n), use this instead:
# best_model_path_str = "/content/caries_detection/run_optimized_yolov8n/weights/best.pt"

best_model_path = Path(best_model_path_str)

if best_model_path.exists():
    print(f">>> Success! Optimized model found at: {best_model_path_str}")
    files.download(best_model_path_str)
    print(">>> Download initiated.")
else:
    print(f"XXX ERROR: Could not find model at {best_model_path_str}")
    print("Please check the run name in Cell 5.")


>>> [Step 6/6] Starting download of the OPTIMIZED 'best.pt'...
>>> Success! Optimized model found at: /content/caries_detection/run_optimized_yolov8s/weights/best.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

>>> Download initiated.
